In [109]:
import os, re
import requests

# --------- Config (env) ----------
BREVO_API_KEY      = os.getenv("BREVO_API_KEY")          # e.g. xkeysib-...
BREVO_TEMPLATE_ID  = int(os.getenv("BREVO_TEMPLATE_ID", "2"))  # your Brevo TEMPLATE_ID
MAILGUN_API_KEY    = os.getenv("MAILGUN_API_KEY")        # e.g. key-xxxxxxxx
MAILGUN_DOMAIN     = os.getenv("MAILGUN_DOMAIN", "for.vdsai.se")
MAILGUN_TEMPLATE   = os.getenv("MAILGUN_TEMPLATE_NAME", "outreach_v1")
MAILGUN_API_BASE   = os.getenv("MAILGUN_API_BASE", "https://api.eu.mailgun.net")  # EU region

# --------- Helpers ----------
def fetch_brevo_template_html(template_id: int):
    """
    Brevo API: GET /v3/smtp/templates/{id}
    Returns (subject, htmlContent)
    """
    url = f"https://api.brevo.com/v3/smtp/templates/{template_id}"
    r = requests.get(url, headers={"api-key": BREVO_API_KEY, "accept": "application/json"})
    r.raise_for_status()
    data = r.json()
    subject = data.get("subject", "")
    html = data.get("htmlContent", "")
    return subject, html

subject, html = fetch_brevo_template_html(3)

In [ ]:
import os, requests, json

def send_mailgun_message(to: list, template: list):
    """
    template list : [name (str), params (dict)]
    """
    
    API = "https://api.eu.mailgun.net"       
    KEY = os.getenv("MAILGUN_API_KEY")   
    DOMAIN = "for.vdsai.se"
    temp_name, temp_params = template
    
    data = {
        "from": "Vikstrand Deep Solutions <info@vdsai.se>",
        "to": to,
        "template": temp_name,
        "t:variables": json.dumps(temp_params),
    }
    r = requests.post(f"{API}/v3/{DOMAIN}/messages", auth=("api", KEY), data=data, timeout=20)
    r.raise_for_status()


    
API = "https://api.eu.mailgun.net"       
KEY = os.getenv("MAILGUN_API_KEY")   
DOMAIN = "for.vdsai.se"
VARS = {
        "nameFirst": "DavidDavidDavidDavid",
        "aNum": "000"
    }

data = {
    "from": "Vikstrand Deep Solutions <info@vdsai.se>",
    "to": ["david.vikstrand@gmail.com"],
    "template": "v2",
    "t:variables": json.dumps(VARS),
}

r = requests.post(f"{API}/v3/{DOMAIN}/messages", auth=("api", KEY), data=data, timeout=20)
r.raise_for_status()
print(r.json())

{'id': '<20250919164049.0caca8aaf09f1644@for.vdsai.se>', 'message': 'Queued. Thank you.'}


In [134]:
import os, csv, requests
from datetime import datetime, timezone
from email.utils import format_datetime

# ----------------- Config -----------------
API_BASE = os.getenv("MAILGUN_API_BASE", "https://api.eu.mailgun.net")
DOMAIN   = os.getenv("MAILGUN_DOMAIN", "for.vdsai.se")
KEY      = os.getenv("MAILGUN_API_KEY")
AUTH     = ("api", KEY)

ACCUM_DIR    = "data/email"
EMAILS_PATH  = os.path.join(ACCUM_DIR, "emails.csv")
STATS_PATH   = os.path.join(ACCUM_DIR, "stats.csv")

# ----------------- Helpers -----------------
def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)

def rfc2822(dt: datetime) -> str:
    return format_datetime(dt.astimezone(timezone.utc))

def fetch_events_single_page(event: str, begin_s: str, end_s: str,
                             limit: int = 100,
                             extra: dict | None = None):
    """
    Single GET (no pagination), returns up to `limit` events.
    NOTE: We do NOT filter by tag here (per your request).
    """
    url = f"{API_BASE}/v3/{DOMAIN}/events"
    params = {"event": event, "begin": begin_s, "end": end_s, "limit": min(limit, 100)}
    if extra: params.update(extra)
    resp = requests.get(url, auth=AUTH, params=params, timeout=30)
    if resp.status_code == 401:
        raise SystemExit("Unauthorized: check Private API key (secret) and EU base URL.")
    resp.raise_for_status()
    return resp.json().get("items", [])

# ==========================================================
#                PER-RECIPIENT emails.csv
# ==========================================================
_STATUS_ORDER = {
    "complained": 7,
    "failed_permanent": 6,
    "dropped": 5,
    "rejected": 4,
    "failed_temporary": 3,
    "clicked": 2,
    "opened": 1,
    "delivered": 0,
}

def _pick_higher(a: str | None, b: str | None) -> str | None:
    if a is None: return b
    if b is None: return a
    return a if _STATUS_ORDER[a] >= _STATUS_ORDER[b] else b

def _touch(rec: dict, evt: dict, status: str):
    ts = evt.get("timestamp")
    ds = evt.get("delivery-status") or {}
    msg_id = ((evt.get("message") or {}).get("headers") or {}).get("message-id")

    rec["status"] = _pick_higher(rec.get("status"), status)

    if rec.get("first_seen") is None or (ts is not None and ts < rec["first_seen"]):
        rec["first_seen"] = ts
    if rec.get("last_seen") is None or (ts is not None and ts > rec["last_seen"]):
        rec["last_seen"] = ts

    if ds.get("code"):    rec["smtp_code"] = ds.get("code")
    if ds.get("message"): rec["smtp_message"] = ds.get("message")
    if msg_id:            rec["message_id"] = msg_id

def compute_email_rows_for_day(day_utc: datetime, *, tag_label: str | None = None) -> list[dict]:
    """
    Build per-recipient status for the UTC day (single-page fetches, ≤100 per type).
    Does NOT filter by tag at the API; only labels rows with tag_label.
    """
    day = day_utc.date()
    begin = datetime(day.year, day.month, day.day, 0, 0, 0, tzinfo=timezone.utc)
    end   = datetime(day.year, day.month, day.day, 23, 59, 59, tzinfo=timezone.utc)
    bs, es = rfc2822(begin), rfc2822(end)

    delivered  = fetch_events_single_page("delivered", bs, es)
    opened     = fetch_events_single_page("opened",    bs, es)
    clicked    = fetch_events_single_page("clicked",   bs, es)
    failed_p   = fetch_events_single_page("failed",    bs, es, extra={"severity": "permanent"})
    failed_t   = fetch_events_single_page("failed",    bs, es, extra={"severity": "temporary"})
    dropped    = fetch_events_single_page("dropped",   bs, es)
    rejected   = fetch_events_single_page("rejected",  bs, es)
    complained = fetch_events_single_page("complained",bs, es)

    agg: dict[tuple[str, str], dict] = {}
    def rec_for(rcpt: str):
        key = (rcpt, tag_label or "")
        if key not in agg:
            agg[key] = {
                "date_utc": day.strftime("%Y-%m-%d"),
                "tag": tag_label or "",
                "recipient": rcpt,
                "status": None,
                "smtp_code": "",
                "smtp_message": "",
                "message_id": "",
                "first_seen": None,
                "last_seen": None,
            }
        return agg[key]

    for e in complained:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "complained")
    for e in failed_p:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "failed_permanent")
    for e in dropped:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "dropped")
    for e in rejected:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "rejected")
    for e in failed_t:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "failed_temporary")
    for e in clicked:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "clicked")
    for e in opened:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "opened")
    for e in delivered:
        if e.get("recipient"): _touch(rec_for(e["recipient"]), e, "delivered")

    rows = []
    for rec in agg.values():
        rec["status"] = rec["status"] or "unknown"
        rec["first_seen"] = "" if rec["first_seen"] is None else str(rec["first_seen"])
        rec["last_seen"]  = "" if rec["last_seen"] is None else str(rec["last_seen"])
        rows.append(rec)
    return rows

def upsert_emails_csv(rows: list[dict], path: str = EMAILS_PATH):
    ensure_dir(os.path.dirname(path))
    exists = os.path.exists(path)
    fieldnames = ["date_utc","tag","recipient","status","smtp_code","smtp_message",
                  "message_id","first_seen","last_seen"]

    if not exists:
        with open(path, "w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=fieldnames)
            w.writeheader()

    existing = []
    with open(path, newline="", encoding="utf-8") as f:
        r = csv.DictReader(f)
        for row in r:
            existing.append(row)

    idx = {(row["date_utc"], row["tag"], row["recipient"]): i for i, row in enumerate(existing)}

    for rec in rows:
        key = (rec["date_utc"], rec["tag"], rec["recipient"])
        if key in idx:
            existing[idx[key]] = {k: str(rec.get(k,"")) for k in fieldnames}
        else:
            existing.append({k: str(rec.get(k,"")) for k in fieldnames})

    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(existing)

    print(f"Upserted {len(rows)} per-recipient rows → {path}")

# ==========================================================
#                 DAILY STATS stats.csv
# ==========================================================
def compute_day_stats(day_utc: datetime, *, tag_label: str | None = None) -> dict:
    """
    Counts for the UTC day (00:00..23:59:59), including delivered and delivery_rate.
    Does NOT filter by tag at the API; only labels the row with tag_label.
    """
    day = day_utc.date()
    begin = datetime(day.year, day.month, day.day, 0, 0, 0, tzinfo=timezone.utc)
    end   = datetime(day.year, day.month, day.day, 23, 59, 59, tzinfo=timezone.utc)
    bs, es = rfc2822(begin), rfc2822(end)

    failed_all  = fetch_events_single_page("failed",   bs, es, limit=100)
    failed_perm = fetch_events_single_page("failed",   bs, es, limit=100, extra={"severity":"permanent"})
    failed_temp = [e for e in failed_all if (e.get("severity") == "temporary")]

    dropped   = fetch_events_single_page("dropped",   bs, es, limit=100)
    rejected  = fetch_events_single_page("rejected",  bs, es, limit=100)
    delivered = fetch_events_single_page("delivered", bs, es, limit=100)

    not_delivered_total = len(failed_perm) + len(failed_temp) + len(dropped) + len(rejected)
    delivered_count     = len(delivered)
    denom = delivered_count + not_delivered_total
    delivery_rate = (delivered_count / denom) if denom > 0 else 0.0

    return {
        "date_utc": day.strftime("%Y-%m-%d"),
        "tag": tag_label or "",
        "failed_permanent": len(failed_perm),
        "failed_temporary": len(failed_temp),
        "dropped": len(dropped),
        "rejected": len(rejected),
        "delivered": delivered_count,
        "not_delivered_total": not_delivered_total,
        "delivery_rate": f"{delivery_rate:.4f}",  # 0–1; format to 4 dp
    }

def _stats_existing_keys(csv_path: str) -> set[tuple[str, str]]:
    """
    Return set of (date_utc, tag) already present to avoid duplicates.
    """
    if not os.path.exists(csv_path):
        return set()
    keys = set()
    with open(csv_path, newline="", encoding="utf-8") as f:
        r = csv.DictReader(f)
        for row in r:
            keys.add((row.get("date_utc",""), row.get("tag","")))
    return keys

def append_stats_row(csv_path: str, row: dict):
    """
    Append row to accumulated stats CSV, creating header if needed; skip if duplicate (date_utc+tag).
    """
    ensure_dir(os.path.dirname(csv_path))
    exists = os.path.exists(csv_path)
    keyset = _stats_existing_keys(csv_path)
    key = (row["date_utc"], row["tag"])
    if key in keyset:
        print(f"Skipped stats (already present): date={row['date_utc']} tag='{row['tag']}'")
        return

    fieldnames = [
        "date_utc", "tag",
        "failed_permanent", "failed_temporary",
        "dropped", "rejected",
        "delivered",
        "not_delivered_total",
        "delivery_rate",
    ]

    with open(csv_path, "a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        if not exists:
            w.writeheader()
        w.writerow(row)
    print(f"Appended stats → {csv_path}: {row}")

# ----------------- Example run (today UTC) -----------------
if __name__ == "__main__":
    if not KEY:
        raise SystemExit("Set MAILGUN_API_KEY (private secret).")

    # Put any label you want here; it will only be written to CSV (not used in API filters)
    TAG_LABEL = "outreach_sep19"

    today_utc = datetime.now(timezone.utc)

    # 1) Per-recipient log
    per_email_rows = compute_email_rows_for_day(today_utc, tag_label=TAG_LABEL)
    upsert_emails_csv(per_email_rows, EMAILS_PATH)

    # 2) Daily aggregate stats (with delivered + delivery_rate)
    stats_row = compute_day_stats(today_utc, tag_label=TAG_LABEL)
    append_stats_row(STATS_PATH, stats_row)


Upserted 2 per-recipient rows → data/email\emails.csv
Skipped stats (already present): date=2025-09-19 tag='outreach_sep19'
